In [2]:
import json
import random
from glob import glob
from tqdm import tqdm

In [3]:
import re

pattern = re.compile(r"'(.+?)' atau '(.+?)'", re.IGNORECASE)

rejected = [
    'help.openai.com',
    'openassistant'
]

break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'model bahasa AI',
    'mempunyai kepercayaan atau pendapat peribadi',
    'tidak mempunyai pendapat peribadi',
    'tidak mempunyai kepercayaan',
    'tidak mempunyai falsafah peribadi',
    'tidak mempunyai pengalaman peribadi',
    'tidak mempunyai pendapat atau pengalaman peribadi',
    'tidak mempunyai maklumat terkini',
    'tidak mempunyai emosi peribadi',
    'tidak mempunyai keutamaan',
    'saya tidak mempunyai akses',
    'tidak mempunyai pengalaman',
    'saya tidak mempunyai keupayaan',
    'tidak mempunyai keupayaan',
    'tidak mempunyai hubungan',
    'tidak mempunyai maklumat',
    'saya tidak mempunyai',
    'saya tidak pernah',
    'saya tidak dapat memahami jawapan',
    '=====',
    '-----',
    'tidak faham bahasa melayu',
    'tidak faham bahasa inggeris',
    'not understand malay',
    'not understand english',
    't understand malay',
    't understand english',
]

break_at_terjemah = [
    'terjemah',
    'translate'
]

rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas',
    'pertama-tama',
    ' akkan',
    'им',
    'м'
]

cyrillic_characters = [
    # Basic Cyrillic Alphabet
    'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ё', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я',
    'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я',

    # Extended Cyrillic Characters
    'Ѐ', 'Ђ', 'Ѓ', 'Є', 'Ѕ', 'І', 'Ї', 'Ј', 'Љ', 'Њ', 'Ћ', 'Ќ', 'Ѝ', 'Ў', 'Џ', 'Ѡ', 'Ѣ', 'Ѥ', 'Ѧ', 'Ѩ', 'Ѫ', 'Ѭ', 'Ѯ', 'Ѱ', 'Ѳ', 'Ѵ', 'Ҁ', 'Ҋ', 'Ҍ', 'Ҏ', 'Ґ', 'Ғ', 'Ҕ', 'Җ', 'Ҙ', 'Қ', 'Ҝ', 'Ҟ', 'Ҡ', 'Ң', 'Ҥ', 'Ҧ', 'Ҩ', 'Ҫ', 'Ҭ', 'Ү', 'Ұ', 'Ҳ', 'Ҵ', 'Ҷ', 'Ҹ', 'Һ', 'Ҽ', 'Ҿ', 'Ӏ', 'Ӂ', 'Ӄ', 'Ӆ', 'Ӈ', 'Ӊ', 'Ӌ', 'Ӎ', 'Ӑ', 'Ӓ', 'Ӕ', 'Ӗ', 'Ә', 'Ӛ', 'Ӝ', 'Ӟ', 'Ӡ', 'Ӣ', 'Ӥ', 'Ӧ', 'Ө', 'Ӫ', 'Ӭ', ' Ӯ', 'Ӱ', 'Ӳ', 'Ӵ', 'Ӷ', 'Ӹ', 'Ӻ', 'Ӽ', 'Ӿ', 'ӿ', 'Ԁ', 'Ԃ', 'Ԅ', 'Ԇ', 'Ԉ', 'Ԋ', 'Ԍ', 'Ԏ', 'Ԑ', 'Ԓ', 'Ԕ', 'Ԗ', 'Ԙ', 'Ԛ', 'Ԝ', 'Ԟ', 'Ԡ', 'Ԣ', 'ԥ', 'Ԧ', 'Ԩ', 'Ԫ', 'Ԭ', 'Ԯ', '԰', 'Բ', 'Դ', 'Զ', 'Ը', 'Ժ', 'Լ', 'Ծ',
    'ѐ', 'ђ', 'ѓ', 'є', 'ѕ', 'і', 'ї', 'ј', 'љ', 'њ', 'ћ', 'ќ', 'ѝ', 'ў', 'џ', 'ѡ', 'ѣ', 'ѥ', 'ѧ', 'ѩ', 'ѫ', 'ѭ', 'ѯ', 'ѱ', 'ѳ', 'ѵ', 'ҁ', 'ҋ', 'ҍ', 'ҏ', 'ґ', 'ғ', 'ҕ', 'җ', 'ҙ', 'қ', 'ҝ', 'ҟ', 'ҡ', 'ң', 'ҥ', 'ҧ', 'ҵ', 'ҫ', 'ҭ', 'ү', 'ұ', 'ҳ', 'ҵ', 'җ', 'ҹ', 'һ', 'ҽ', 'ҿ', 'ӏ', 'ӂ', 'ӄ', 'ӆ', 'ӈ', 'ӊ', 'ӌ', 'ӎ', 'ạ', 'ӓ', 'ӕ', 'ӗ', 'ә', 'ӛ', 'ӝ', 'ӟ', 'ӡ', 'ӣ', 'ӥ', 'ӧ', 'ө', 'ӫ', 'ӭ', 'ӯ', 'ӱ', 'ӳ', 'ӵ', 'ғ', 'ӷ', 'ӹ', 'ӻ', 'ӽ', 'ӿ', 'ԁ', 'ԃ', 'ԅ', 'ԇ', 'ԉ', 'ԋ', 'ԍ', 'ԏ', 'ԑ', 'ԓ', 'ԕ', 'ԗ', 'ԙ', 'ԛ', 'ԝ', 'ԟ', 'ԡ', 'ԣ', 'ԥ', 'ԧ', 'ԩ', 'ԫ', 'ԭ', 'ԯ', 'Ա', 'Գ', 'Ե', 'Է', 'Թ', 'Ի', 'Խ', 'Կ'
]

cyrillic_characters = set(cyrillic_characters)

def found_word(s, words):
    for i in range(len(words)):
        if words[i] in s:
            return True, words[i]
    return False, None

def detect_ngram_repetitions(text, n=10):
    tokens = text.split()
    ngrams = defaultdict(int)

    for i in range(len(tokens) - n + 1):
        ngram = ' '.join(tokens[i:i+n])
        ngrams[ngram] += 1
    repeated_ngrams = {ngram: count for ngram, count in ngrams.items() if count > 1}
    
    return repeated_ngrams

indons = []

def accept(d, min_len = 10, skip_indon = True, skip_translation = True):
    global indons
    
    d = d.strip()
    
    if len(d.split()) < min_len:
        return False
    
    if len(set(d) & cyrillic_characters):
        return False
        
    if 'terjemahkan teks' in d.lower():
        return False

    if 'no need to translate' in d.lower():
        return False

    if 'can be translated' in d.lower():
        return False

    if 'cannot translate' in d.lower():
        return False

    if 'should be translated to' in d.lower():
        return False

    if 'cannot be translated' in d.lower():
        return False

    if 'standard malay' in d.lower():
        return False

    if 'would not be translated' in d.lower():
        return False

    if 'as an AI language model' in d:
        return False

    if 'should be translated as' in d.lower():
        return False

    if 'Bahasa Malaysia Standard' in d:
        return False
    
    if 'Saya adalah model AI' in d:
        return False
    
    if 'saya model AI' in d:
        return False
    
    if 'Saya model AI' in d:
        return False
    
    if 'sebagai model AI' in d:
        return False
    
    if 'Sebagai model AI' in d:
        return False
    
    if 'model bahasa AI' in d:
        return False
    
    if 'model AI yang dibangunkan' in d:
        return False
    
    if d == '<s>':
        return False
    
    if 'tidak dapat memberikan maklumat' in d:
        return False
    
    if 'Sebagai model bahasa' in d:
        return False
    
    match = pattern.search(d)
    if match:
        return False
    
    d_lower = d.lower()
    splitted = d_lower.split()
    if (len(set(splitted)) / len(splitted)) < 0.2:
        return False
    
    words = d.split()
    ratio_words = [w for w in words if len(w) > 100 and (len(set(w)) / len(w)) <= 0.2]
    if len(ratio_words):
        return False
    
    repeated = detect_ngram_repetitions(d, n = 10)
    for v in repeated.values():
        if v > 2:
            return False
    
    if found_word(d_lower, rejected)[0]:
        return False
    
    if found_word(d_lower, break_at)[0]:
        return False
    
    if skip_translation and found_word(d_lower, break_at_terjemah)[0]:
        return False
    
    if skip_indon:
        found_indon = found_word(d_lower, rejected_words)
        if found_indon[0]:
            indons.append((d, found_indon[1]))
            return False
    
    return True

In [4]:
from collections import defaultdict
from tqdm import tqdm
import numpy as np

with open('post-postfilter.jsonl', 'w') as fopen_l:

    with open('postfilter.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)

            if '```' not in l['answer_ms']:
                continue

            if l['instruction_ms'] is None:
                continue
            else:
                q = l['instruction_ms']
                if len(set(q.split())) < (len(set(l['instruction'].split())) / 2):
                    continue

            if l['answer_ms'] is None:
                continue

            if len(set(l['answer_ms'].split())) < (len(set(l['answer'].split())) / 2):
                continue

            if l['answer'] is None:
                continue

            try:
                
                lang = l['answer_ms'].split('```')[1].split('\n')[0]
                if 'cudaMalloc' in lang:
                    continue

                if not accept(l['answer_ms']):
                    continue

                if len(l['answer_ms'].split()) / len(l['answer'].split()) > 1.1:
                    continue

                if len(l['answer_ms'].split()) / len(l['answer'].split()) < 0.91:
                    continue

                splitted = l['answer_ms'].split()
                ms = len(set(splitted)) / len(splitted)
                splitted = l['answer'].split()
                en = len(set(splitted)) / len(splitted)

                if np.abs(en - ms) > 0.15:
                    continue

                if l['answer'].strip().endswith('```') and not l['answer_ms'].strip().endswith('```'):
                    continue
                
                fopen_l.write(f'{json.dumps(l)}\n')
                fopen_l.flush()
            except:
                pass

219018it [01:14, 2935.60it/s]


In [5]:
!wc -l post-postfilter.jsonl

132352 post-postfilter.jsonl


In [6]:
!ls -lh post-postfilter.jsonl

-rw-rw-r-- 1 husein husein 856M Sep  30 23:30 post-postfilter.jsonl


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='post-postfilter.jsonl',
    path_in_repo="post-postfilter.jsonl",
    repo_id="mesolitica/mixtral-magicoder",
    repo_type="dataset",
)

post-postfilter.jsonl:   0%|          | 0.00/897M [00:00<?, ?B/s]